In [136]:
import numpy as np
from matplotlib import pyplot as plt
import time
import torch
from torch import nn
import os

## Objective Funtions

In [312]:
def sphere(x):
    return torch.sum(x**2)

In [138]:
def ellipsoid(x):
    n = x.size(0)
    out = 0
    for i in range(n):
        out += x[i]**2 * 10**(6 * i / (n-1))
    return out

In [139]:
def cigar(x):
    return x[0]**2 + 10**6 * torch.sum(x[1:]**2)

In [140]:
def tablet(x):
    return 10**6 * x[0]**2 + torch.sum(x[1:]**2)

In [141]:
def parabolic_ridge(x):
    return -x[0] + 100*torch.sum(x[1:]**2)

In [142]:
def sharp_ridge(x):
    return -x[0] + 100*torch.norm(x[1:])

In [143]:
def diffpow(x):
    n = x.size(0)
    out = 0
    for i in range(n):
        out += torch.abs(x[i])**(2 + 10*i/(n-1))
    return out

In [144]:
def rosenbrock(x):
    n = x.size(0)
    out = 0
    for i in range(n-1):
        out += 100*(x[i+1] - x[i]**2)**2 + (x[i] - 1)**2
    return out

In [145]:
#NOTE: Only takes x in R^2
def booth(x):
    return (x[0] + 2*x[1] - 7)**2 + (2*x[0] + x[1] - 5)**2

In [146]:
#TODO: Add more

# Gradient-Based Optimization Methods

Source: https://algorithmsbook.com/optimization/files/optimization.pdf

In [93]:
#Gradient Descent
def gd_step(x, obj, params):
    lr = params["lr"]
    
    loss = obj(x)
    grad, = torch.autograd.grad(loss, inputs=x)
    x_new = x - lr*grad
    return x_new

In [185]:
#Momentum
def momentum_step(x, obj, params):
    lr = params["lr"]
    v = params["v"]
    decay = params["decay"]
    
    loss = obj(x)
    grad, = torch.autograd.grad(loss, inputs=x)
    v_new = decay*v - lr*grad
    x_new = x + v_new
    return x_new, v_new

In [196]:
#AdaGrad
def adagrad_step(x, obj, params):
    lr = params["lr"]
    s = params["s"]
    
    loss = obj(x)
    grad, = torch.autograd.grad(loss, inputs=x)
    s_new = s + grad**2
    x_new = x - (lr / (10e-8 + torch.sqrt(s_new)))*grad
    return x_new, s_new

In [96]:
#RMSProp

In [364]:
#Adam
def adam_step(x, obj, params):
    v = params["v"]
    v_decay = params["v_decay"]
    s = params["s"]
    s_decay = params["s_decay"]
    lr = params["lr"]
    k = params["k"] #stores number of iterations
    
    k += 1
    
    loss = obj(x)
    grad, = torch.autograd.grad(loss, inputs=x)
    
    v_new = v_decay*v + (1-v_decay)*grad
    s_new = s_decay*s + (1-s_decay)*(grad**2)
    
    v_hat = v_new / (1 - v_decay**k)
    s_hat = s_new / (1 - s_decay**k)
    
    x_new = x - lr * v_hat / (1e-8 + torch.sqrt(s_hat))
    return x_new, v_new, s_new, k

## CMA-ES

Source: http://www.cmap.polytechnique.fr/~nikolaus.hansen/evco_11_1_1_0.pdf

In [325]:
def oldcmaes_step(x, obj, params):
    l = params["l"]
    mu = params["mu"]
    C = params["C"]
    pc = params["pc"]
    cc = params["cc"]
    ccov = params["ccov"]
    s = params["s"]
    ps = params["ps"]
    cs = params["cs"]
    ds = params["ds"]
    chi = params["chi"]
    n = x.size(0)
    
    #compute B and D
    B, D_squared, B_transpose = torch.linalg.svd(C)
    D = torch.diag(torch.sqrt(D_squared))
    BD = torch.matmul(B,D)
    
    #Sample offspring
    m = torch.distributions.multivariate_normal.MultivariateNormal(torch.zeros(n), torch.eye(n))
    z = m.sample((l,))
    offspring = x + s * torch.t(torch.matmul(BD, torch.t(z)))
        
    #Compute objective on each offspring
    evals = torch.zeros(l)
    for i in range(l):
        evals[i] = obj(offspring[i])
        
    #Get indices of top mu offspring
    top_evals, top_inds = torch.topk(evals, mu, largest=False)
    
    #Updates
    x_new = (1/mu) * torch.sum(offspring[top_inds], dim=0)
    z_avg = (1/mu) * torch.sum(z[top_inds], dim=0)
    
    #Covariance update
    pc_new = (1-cc)*pc + np.sqrt(cc*(2-cc)*mu) * torch.matmul(BD, z_avg)
    C_new = (1-ccov)*C + ccov*torch.outer(pc_new, pc_new)
    
    #Step size update
    ps_new = (1-cs)*ps + np.sqrt(cs*(2-cs)*mu) * torch.matmul(B, z_avg)
    s_new = s * torch.exp((1/ds) * (torch.norm(ps_new)-chi) / chi)
    
    return x_new, C_new, pc_new, s_new, ps_new

Source: http://www.cmap.polytechnique.fr/~nikolaus.hansen/evco_11_1_1_0.pdf

In [352]:
def newcmaes_step(x, obj, params):
    l = params["l"]
    mu = params["mu"]
    C = params["C"]
    pc = params["pc"]
    cc = params["cc"]
    ccov = params["ccov"]
    acov = params["acov"]
    s = params["s"]
    ps = params["ps"]
    cs = params["cs"]
    ds = params["ds"]
    chi = params["chi"]
    n = x.size(0)
    
    #compute B and D
    B, D_squared, B_transpose = torch.linalg.svd(C)
    D = torch.diag(torch.sqrt(D_squared))
    BD = torch.matmul(B,D)
    
    #Sample offspring
    m = torch.distributions.multivariate_normal.MultivariateNormal(torch.zeros(n), torch.eye(n))
    z = m.sample((l,))
    offspring = x + s * torch.t(torch.matmul(BD, torch.t(z)))
        
    #Compute objective on each offspring
    evals = torch.zeros(l)
    for i in range(l):
        evals[i] = obj(offspring[i])
        
    #Get indices of top mu offspring
    top_evals, top_inds = torch.topk(evals, mu, largest=False)
    
    #Updates
    x_new = (1/mu) * torch.sum(offspring[top_inds], dim=0)
    z_avg = (1/mu) * torch.sum(z[top_inds], dim=0)
    
    #Covariance update
    pc_new = (1-cc)*pc + np.sqrt(cc*(2-cc)*mu) * torch.matmul(BD, z_avg)
    
    outer_prod_sum = torch.zeros(n,n)
    for i in range(mu):
        outer_prod_sum += (1/mu) * torch.outer(z[top_inds[i]], z[top_inds[i]])
    bigZ = torch.matmul(BD, torch.matmul(outer_prod_sum, torch.t(BD)))
    
    C_new = ((1-ccov)*C + 
             ccov * (acov*torch.matmul(pc_new, torch.t(pc_new)) +
                     (1-acov)*bigZ))
    
    #Step size update
    ps_new = (1-cs)*ps + np.sqrt(cs*(2-cs)*mu) * torch.matmul(B, z_avg)
    s_new = s * torch.exp((1/ds) * (torch.norm(ps_new)-chi) / chi)
    
    return x_new, C_new, pc_new, s_new, ps_new

Source: https://arxiv.org/pdf/1604.00772.pdf (see pg. 29)

In [468]:
#TODO - finish hs
def cmaes_step(x, obj, params):
    #Set parameters
    l = params["l"]
    mu = params["mu"]
    mu_eff = params["mu_eff"]
    C = params["C"]
    pc = params["pc"]
    cc = params["cc"]
    cm = params["cm"]
    c1 = params["c1"]
    cmu = params["cmu"]
    s = params["s"]
    ps = params["ps"]
    cs = params["cs"]
    ds = params["ds"]
    chi = params["chi"]
    w = params["w"]
    sumw = params["sumw"]
    n = x.size(0)
    
    #Calculate B,D
    B, D_squared_diag, B_transpose = torch.linalg.svd(C)
    D_diag = torch.sqrt(D_squared_diag)
    D = torch.diag(D_diag)
    Dinv = torch.diag(1/D_diag)
    BD = torch.matmul(B,D)
    sqrtCinv = torch.matmul(B, torch.matmul(Dinv, B_transpose))
    
    #Sample offspring
    m = torch.distributions.multivariate_normal.MultivariateNormal(torch.zeros(n), torch.eye(n))
    z = m.sample((l,))
    y = torch.matmul(BD, z)
    offspring = x + s*y
    
    #Compute objective on each offspring
    evals = torch.zeros(l)
    for i in range(l):
        evals[i] = obj(offspring[i])
    
    #Get indices of top mu offspring
    top_evals, top_inds = torch.topk(evals, l, largest=False)
    
    #Selection and recombination
    y_avg = torch.zeros(n)
    for i in range(mu):
        y_avg += w[i] * y[top_inds[i]]
    x_new = x + cm*s*y_avg

    #Covariance update    
    hs = 0 #Annoying
    dhs = (1-hs) * cc * (2-cc)
    
    w_circle = torch.zeros(l)
    w_circle[:mu] = w[:mu]
    for i in range(mu, l):
        w_circle[i] = w[i] * n / torch.norm(torch.matmul(sqrtCinv, y[top_inds[i]]))**2
    
    pc_new = (1-cc)*pc + hs*np.sqrt(cc*(2-cc)*mu_eff)*y_avg

    outer_prod_sum = torch.zeros(n,n)
    for i in range(l):
        outer_prod_sum += w_circle[i] * torch.outer(y[top_inds[i]], y[top_inds[i]])
    
    C_new = ((1 + c1*dhs - c1 - cmu*sumw)*C +
         c1*torch.outer(pc, pc) +
         cmu*outer_prod_sum)
    
    #Step size update
    ps_new = (1-cs)*ps + np.sqrt(cs*(2-cs)*mu_eff) * torch.matmul(sqrtCinv, y_avg)
    s_new = s * torch.exp((cs/ds) * (torch.norm(ps)/chi - 1))
    
    return x_new, C_new, pc_new, s_new, ps_new

## Optimizer Wrapper

In [455]:
def step(optimizer, x, obj):
    if optimizer == "gd":
        x_new = gd_step(x, obj, parameters["gd"])
        
    elif optimizer == "momentum":
        x_new, v_new = momentum_step(x, obj, parameters["momentum"])
        parameters["momentum"]["v"] = v_new
        
    elif optimizer == "adagrad":
        x_new, s_new = adagrad_step(x, obj, parameters["adagrad"])
        parameters["adagrad"]["s"] = s_new
        
    elif optimizer == "rmsprop":
        pass
    
    elif optimizer == "adam":
        x_new, v_new, s_new, k = adam_step(x, obj, parameters["adam"])
        parameters["adam"]["v"] = v_new
        parameters["adam"]["s"] = s_new
        parameters["adam"]["k"] = k
    
    elif optimizer == "oldcmaes":
        x_new, C_new, pc_new, s_new, ps_new = oldcmaes_step(x, obj, parameters["oldcmaes"])
        parameters["oldcmaes"]["C"] = C_new
        parameters["oldcmaes"]["pc"] = pc_new
        parameters["oldcmaes"]["s"] = s_new
        parameters["oldcmaes"]["ps"] = ps_new
        
    elif optimizer == "newcmaes":
        x_new, C_new, pc_new, s_new, ps_new = newcmaes_step(x, obj, parameters["newcmaes"])
        parameters["newcmaes"]["C"] = C_new
        parameters["newcmaes"]["pc"] = pc_new
        parameters["newcmaes"]["s"] = s_new
        parameters["newcmaes"]["ps"] = ps_new
        
    elif optimizer == "cmaes":
        x_new, C_new, pc_new, s_new, ps_new = cmaes_step(x, obj, parameters["cmaes"])
        parameters["cmaes"]["C"] = C_new
        parameters["cmaes"]["pc"] = pc_new
        parameters["cmaes"]["s"] = s_new
        parameters["cmaes"]["ps"] = ps_new
    
    else:
        print("Optimizer", optimizer, "not found.")
        
    return x_new

# Parameters

In [494]:
def set_params(n):
    gd_params = {
        "lr": 0.0001
    }

    momentum_params = {
        "lr": 0.001,
        "v": 0,
        "decay": 0.5
    }

    adagrad_params = {
        "lr": 1,
        "s": 0
    }

    rmsprop_params = {}

    adam_params = {
        "lr": 0.3,
        "v": 0,
        "s": 0,
        "v_decay": 0.9,
        "s_decay": 0.999,
        "k": 0
    }
    
    #OLD/NEW CMAES PARAMETERS
    l = int(4 + np.floor(3*np.log(n)))
    mu = int(np.floor(l/4))
    chi = np.sqrt(n) * (1 - 1/(4*n) + 1/(21*n**2))
    acov = 1/mu
    
    oldcmaes_params = {
        "l": l,
        "mu": mu,
        "C": torch.eye(n),
        "pc": torch.zeros(n),
        "cc": 4 / (n+4),
        "ccov": 2 / (n + np.sqrt(2))**2,
        "s": 1,
        "ps": torch.zeros(n),
        "cs": 4/(n+4),
        "ds": 1 + (n+4)/4,
        "chi": chi
    }
    
    newcmaes_params = {
        "l": l,
        "mu": mu,
        "C": torch.eye(n),
        "pc": torch.zeros(n),
        "cc": 4 / (n+4),
        "ccov": acov*2/(n+np.sqrt(2))**2 + (1-acov)*min(1, (2*mu-1)/((n+2)**2 + mu)),
        "acov": acov,
        "s": 1,
        "ps": torch.zeros(n),
        "cs": 4/(n+4),
        "ds": 1 + (n+4)/4,
        "chi": chi
    }
    
    #CMAES PARAMETERS
    l = int(4 + np.floor(3 * np.log(n)))
    mu = int(np.floor(l/2))
    
    w_prime = np.log((l+1)/2) - torch.log(torch.arange(1,l+1))
        
    mu_eff = (torch.sum(w_prime[:mu])**2) / torch.sum(w_prime[:mu]**2)
    mu_eff_neg = (torch.sum(w_prime[mu:])**2) / torch.sum(w_prime[mu:]**2)
    
    cm = 1
    
    cs = (mu_eff + 2) / (n + mu_eff + 5)
    ds = 1 + 2 * max(0, np.sqrt((mu_eff-1)/(n+1))-1) + cs
    
    cc = (4 + mu_eff/n) / (n + 4 + 2*mu_eff/n)
    c1 = 2 / ((n+1.3)**2 + mu_eff)
    cmu = min(1-c1, 2*(1/4 + mu_eff + 1/mu_eff - 2)/((n+2)**2 + mu_eff))
    
    amu = 1 + c1/cmu
    amu_eff = 1 + (2*mu_eff_neg) / (mu_eff + 2)
    a_pd = (1 - c1 - cmu) / (n*cmu)
    
    sum_w_prime_pos = torch.sum(w_prime[:mu])
    sum_w_prime_neg = -torch.sum(w_prime[mu:])
    
    w = torch.zeros(l)
    w[:mu] = w_prime[:mu] / sum_w_prime_pos
    w[mu:] = min(amu, amu_eff, a_pd) * w_prime[mu:] / sum_w_prime_neg
    sumw = torch.sum(w)
    
    chi = np.sqrt(1 - 1/(4*n) + 1/(21*n**2))
    
    cmaes_params = {
        "l": l,
        "mu": mu,
        "mu_eff": mu_eff,
        "C": torch.eye(n),
        "pc": torch.zeros(n),
        "cc": cc,
        "cm": cm,
        "c1": c1,
        "cmu": cmu,
        "s": 0.1,
        "ps": torch.zeros(n),
        "cs": cs,
        "ds": ds,
        "chi": chi,
        "w": w,
        "sumw": sumw
    }

    parameters = {
        "gd": gd_params,
        "momentum": momentum_params,
        "adagrad": adagrad_params,
        "rmsprop": rmsprop_params,
        "adam": adam_params,
        "oldcmaes": oldcmaes_params,
        "newcmaes": newcmaes_params,
        "cmaes": cmaes_params
    }
    
    return parameters

## Training/Tests

In [500]:
epochs = 5001
optimizer = "adam"
obj = rosenbrock
x = torch.arange(start=1,end=6, dtype=torch.float32, requires_grad=True)

print("Initial x:", x)

parameters = set_params(x.size(0))

for epoch in range(epochs):
    if epoch % 500 == 0:
        print("epoch", epoch, "|",obj(x).item())
        #print(x)
    x = step(optimizer, x, obj)

print("Final x:", x)

Initial x: tensor([1., 2., 3., 4., 5.], requires_grad=True)
epoch 0 | 14814.0
epoch 500 | 1.5921648740768433
epoch 1000 | 1.253328561782837
epoch 1500 | 0.8334202766418457
epoch 2000 | 0.4226188659667969
epoch 2500 | 0.13334880769252777
epoch 3000 | 0.019047601148486137
epoch 3500 | 0.5619878768920898
epoch 4000 | 2.4986173229990527e-05
epoch 4500 | 5.5500800954177976e-05
epoch 5000 | 9.697453788248822e-05
Final x: tensor([0.9989, 0.9979, 0.9958, 0.9915, 0.9831], grad_fn=<SubBackward0>)
